In [1]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
from datetime import datetime, timedelta
import pandahouse

token = '7044101108:AAETY0C2dU2iAJAV-N7y497ztoXxCBloq2E'
bot = telegram.Bot(token=token) 

chat_id =  1027001565

connection = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : 'dpo_python_2020', 
    'user' : 'student', 
    'database' : 'simulator'
}

In [79]:
    def extract_data():
        
        query = '''
            SELECT toDate(time) AS event_date,
                   countIf(action='view') AS views,
                   countIf(action='like') AS likes,
                   countIf(action='like')/countIf(action='view') AS CTR,
                   count(DISTINCT user_id) AS unique_users
            FROM simulator_20240620.feed_actions
            GROUP BY toDate(time), country
        '''
        event_df = pandahouse.read_clickhouse(connection=connection, query=query)
        
        query = '''
            SELECT toString(start_day) AS start_day,
                   toString(day) AS day,
                   count(user_id) AS users
               FROM
                 (SELECT *
                  FROM
                    (SELECT user_id,
                            min(toDate(time)) AS start_day
                     FROM simulator_20240620.feed_actions
                     GROUP BY user_id) t1
                  JOIN
                    (SELECT DISTINCT user_id,
                                     toDate(time) AS day
                     FROM simulator_20240620.feed_actions) t2 USING user_id
                  )
               WHERE toDate(start_day) >= today() - 30
               GROUP BY start_day,
                        day
        '''  
        retention_df = pandahouse.read_clickhouse(connection=connection, query=query)
        
        query = '''
            SELECT toDate(time) AS event_date,
                   user_id,
                   receiver_id
            FROM simulator_20240620.message_actions
        '''
        df_message_actions = pandahouse.read_clickhouse(connection=connection, query=query)
        
        return event_df, retention_df, df_message_actions
event_df, retention_df, df_message_actions = extract_data()

In [78]:
retention_df

,start_day,day,users
0,2024-08-04,2024-08-05,364
1,2024-07-21,2024-08-05,140
2,2024-07-18,2024-07-18,763
3,2024-07-17,2024-07-30,206
4,2024-07-11,2024-07-15,193
...,...,...,...
491,2024-07-07,2024-08-05,119
492,2024-07-12,2024-08-03,51
493,2024-07-08,2024-07-24,227
494,2024-07-24,2024-08-02,269


In [67]:
    def transform_data(event_df, retention_df, df_message_actions):
        wau_df = event_df.set_index('event_date', inplace=False).resample('W').sum().reset_index()
        mau_df = event_df.set_index('event_date', inplace=False).resample('MS').sum().reset_index()
        
        pivoted_retention = retention_df.pivot(index="day", columns="start_day", values="users")
        value_to_divide = retention_df.sort_values('start_day').reset_index()['start_day'][0]
        pivoted_retention = pivoted_retention.divide(pivoted_retention[value_to_divide], axis=0)  
        
        group_by_date = df_message_actions.groupby('event_date', as_index=False).nunique()
        
        return (event_df, wau_df, mau_df, pivoted_retention, group_by_date)

In [68]:
event_df, wau_df, mau_df, pivoted_retention, group_by_date = transform_data(event_df, retention_df, df_message_actions)

In [72]:
    def load_data(event_df, wau_df, mau_df, pivoted_retention, group_by_date):
        # send data
        message = 'Отчет по работе приложения'
        bot.sendMessage(chat_id = chat_id, text = message)
        send_graph(event_df, 'DAU', 'event_date', 'unique_users', 'DAU.png')
        send_graph(wau_df, 'WAU', 'event_date', 'unique_users', 'WAU.png')
        send_graph(mau_df, 'MAU', 'event_date', 'unique_users', 'MAU.png')
        send_heatmap(pivoted_retention)

        message = 'Данные об ленте новостей.'
        bot.sendMessage(chat_id = chat_id, text = message)
        send_graph(event_df, 'Количество просмотров ленты', 'event_date', 'views', 'Количество просмотров ленты.png')
        send_graph(event_df, 'Количество лайков ленты', 'event_date', 'likes', 'Количество лайков ленты.png')    
        send_graph(event_df, 'Значения CTR', 'event_date', 'CTR', 'Значения CTR.png')
        
        # send data
        message = 'Данные об сообщениях.'
        bot.sendMessage(chat_id = chat_id, text = message)
        send_graph(group_by_date, 'Количество пользователей отправляющих сообщение', 'event_date', 'user_id', 'Количество пользователей отправляющих сообщение.png')
        send_graph(group_by_date, 'Количество пользователей принимающих сообщение', 'event_date', 'receiver_id', 'Количество пользователей принимающих сообщение.png')

In [70]:
    def send_graph(data, title, x, y, title_plot):
        # daily active users 
        sns.set(style="whitegrid")
        plt.figure(figsize=(15, 6))
        plt.title(title, fontsize=15)  
        sns.lineplot(data=data, x=x, y=y)
        
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0)
        plot_object.name = title_plot
        plt.close()
        bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [71]:
    def send_heatmap(pivoted_retention):
        plt.figure(figsize=(25, 10))
        sns.heatmap(pivoted_retention, annot=True, fmt='.2f', linewidth=.3)
        
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0)
        plot_object.name = 'Retention.png'
        plt.close()
        bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [73]:
load_data(event_df, wau_df, mau_df, pivoted_retention, group_by_date)